In [1]:
# SparkConf 객체 생성
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

In [2]:
conf = SparkConf()
conf.set("spark.executor.memory", "8g")
conf.set("spark.executor.cores", "4")
conf.set("spark.default.parallelism", "8")
conf.set("spark.sql.shuffle.partitions", "8")
conf.set("spark.memory.storageFraction", "0.9")
conf.set("spark.sql.broadcastTimeout", "300")

In [3]:
spark = SparkSession.builder \
        .appName("read_spark_s3") \
        .config(conf=conf) \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/27 13:42:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/27 13:42:29 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/07/27 13:42:29 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/07/27 13:42:29 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/07/27 13:42:29 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/07/27 13:42:29 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:455)
	at java.base/sun.nio.ch.Net.bind(Net.java:447)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:227)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:141)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext

In [24]:
existing_parquet_path = "s3a://test-ml-storage/ml-test/aizen_de_test_1.parquet"
existing_df = spark.read.parquet(existing_parquet_path)

In [30]:
existing_df.head()

Row(column1=-2.2173403793526587, column2=0.00758673003436465, column3=-1.2281227573926903, column4=-0.37805605338640363, column5=-0.9165316838156667, column6=1.3604948196906261, column7=0.7327649543756789, column8=-0.15288463526921456, column9=1.3719710050836469, column10=0.8186382710212818, column11=-1.252063064818419, column12=0.3562083229488128, column13=-0.3560605544548481, column14=0.44966565816191933, column15=-0.06055706071870122, column16=-1.4569930517925822, column17=0.13881348352529832, column18=-1.2793880719031108, column19=-1.4997871711944482, column20=0.5135078605644979, column21=1.1308727959459342, column22=-0.4057851464043333, column23=0.4352485324856836, column24=1.1253294457081189, column25=-1.3570505408289912, column26=0.21559982522691573, column27=-0.06320074221682982, column28=-0.49218150704591057, column29=0.5221147792549723, column30=1.4568814908264625, column31=-1.5991383847511735, column32=-0.7330810166310215, column33=-0.5795351489437721, column34=-2.3326961611

In [25]:
print(existing_df.count(), len(existing_df.columns))

1002000 102


In [26]:
existing_df.createOrReplaceTempView("ml_test")

In [27]:
sql_query = "SELECT AVG(column1), AVG(column2) FROM ml_test"
result_df = spark.sql(sql_query)

In [28]:
result_df.collect()

[Row(avg(column1)=-0.02092034395542564, avg(column2)=0.0025485385923748947)]

In [29]:
existing_df.write.mode('append').parquet("s3a://test-ml-storage/ml-test/aizen_de_test_1.parquet")

23/07/25 16:19:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/07/25 16:19:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/07/25 16:19:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/07/25 16:19:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/07/25 16:19:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/07/25 16:19:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/07/25 16:19:59 WARN MemoryManager: Total allocation exceeds 95.

In [31]:
spark.stop()